In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

In [4]:
# chemin du fichier CSV
csv_path = "./data/donnees_nicla.csv"

# lecture du fichier CSV avec pandas
df = pd.read_csv(csv_path)

# affichage des 5 premières lignes
df.head()

FileNotFoundError: [Errno 2] No such file or directory: './data/donnees_nicla.csv'

In [ ]:
# récupérer les colonnes dans des "arrays" numpy
count = df['count'].to_numpy()
ax = df['x'].to_numpy()
ay = df['y'].to_numpy()
az = df['z'].to_numpy()
delta_t = df['delta_t'].to_numpy()

In [ ]:
buffer_size = 100

plt.plot(ax[:buffer_size], az[:buffer_size], marker='o')
plt.xlabel('ax')
plt.ylabel('az')
plt.title(f'az en fonction de ax ({buffer_size} premières valeurs)')
plt.grid(True)
plt.show()

NameError: name 'df' is not defined

In [8]:
def plot_data(x, y, start=0, end=None, xlabel='x', ylabel='y', title=None, marker='o'):
    """
    Plot y versus x for a given interval.
    
    Parameters:
        x, y : numpy arrays to plot
        start : index de début (inclus)
        end : index de fin (exclus), None pour la fin du tableau
        xlabel, ylabel : labels des axes
        title : titre du graphique, None pour générer automatiquement
        marker : style des points
    """
    if end is None:
        end = len(x)
    
    plt.plot(x[start:end], y[start:end], marker=marker)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    
    if title is None:
        title = f'{ylabel} en fonction de {xlabel} ({end-start} valeurs)'
    plt.title(title)
    
    plt.grid(True)
    plt.show()

In [10]:
def trapezoidal_integration(data_x, data_y, data_z, delta_t):
    """
    Integrate 3D data using trapezoidal rule.
    
    Parameters:
        data_x, data_y, data_z : numpy arrays of values to integrate
        delta_t : numpy array of time steps (same length as data arrays)
    
    Returns:
        integrated : numpy array of shape (N, 3) with integrated values for x, y, z
    """
    N = len(data_x)
    integrated = np.zeros((N, 3))
    
    for i in range(1, N):
        integrated[i, 0] = integrated[i-1, 0] + 0.5 * (data_x[i] + data_x[i-1]) * delta_t[i]
        integrated[i, 1] = integrated[i-1, 1] + 0.5 * (data_y[i] + data_y[i-1]) * delta_t[i]
        integrated[i, 2] = integrated[i-1, 2] + 0.5 * (data_z[i] + data_z[i-1]) * delta_t[i]

    return integrated


In [9]:
def euler_integration(data_x, data_y, data_z, delta_t):
    """
    Integrate 3D data using simple Euler method.
    
    Parameters:
        data_x, data_y, data_z : numpy arrays of values to integrate
        delta_t : numpy array of time steps (same length as data arrays)
    
    Returns:
        integrated : numpy array of shape (N, 3) with integrated values for x, y, z
    """
    N = len(data_x)
    integrated = np.zeros((N, 3))
    
    for i in range(1, N):
        dt = delta_t[i]
        integrated[i, 0] = integrated[i-1, 0] + data_x[i-1] * dt
        integrated[i, 1] = integrated[i-1, 1] + data_y[i-1] * dt
        integrated[i, 2] = integrated[i-1, 2] + data_z[i-1] * dt
    
    return integrated

In [11]:
vx, vy, vz = trapezoidal_integration(ax, ay, az, delta_t).T

NameError: name 'ax' is not defined

In [12]:
vx, vy, vz = trapezoidal_integration(ax, ay, az, delta_t).T

NameError: name 'ax' is not defined

In [14]:
import numpy as np
from scipy.signal import butter, filtfilt

def high_pass_filter_scipy(data_x, data_y, data_z, cutoff, sample_period, order=1):
    """
    Apply a high-pass Butterworth filter on 3D data.
    
    Parameters:
        data_x, data_y, data_z : numpy arrays of input data
        cutoff : cutoff frequency in Hz
        sample_period : time between samples in seconds
        order : filter order (default 1)
    
    Returns:
        filtered_x, filtered_y, filtered_z : filtered numpy arrays
    """
    fs = 1 / sample_period  # sampling frequency
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    
    filtered_x = filtfilt(b, a, data_x)
    filtered_y = filtfilt(b, a, data_y)
    filtered_z = filtfilt(b, a, data_z)
    
    return filtered_x, filtered_y, filtered_z


In [15]:
import numpy as np

def high_pass_filter_iir(x, y, z, alpha):
    """
    Simple high-pass filter for 3D data, using difference equation
    like it can be done on Nicla Sense ME.
    
    Parameters:
        x, y, z : numpy arrays of input data
        alpha : filter coefficient between 0 and 1 (smaller = plus de filtrage)
    
    Returns:
        x_hp, y_hp, z_hp : filtered arrays
    """
    x_hp = np.zeros_like(x)
    y_hp = np.zeros_like(y)
    z_hp = np.zeros_like(z)
    
    for i in range(1, len(x)):
        x_hp[i] = alpha * (x_hp[i-1] + x[i] - x[i-1])
        y_hp[i] = alpha * (y_hp[i-1] + y[i] - y[i-1])
        z_hp[i] = alpha * (z_hp[i-1] + z[i] - z[i-1])
    
    return x_hp, y_hp, z_hp
